In [1]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [2]:
user_question = 'I just discovered the course. Can I still join it?'
user_question_vector = embedding_model.encode(user_question)

In [3]:
'''
Q1: What's the first value of the resulting vector?
A: 0.07
'''
user_question_vector[0]

0.078222655

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
ml_documents = list(filter(lambda x: x['course'] == 'machine-learning-zoomcamp', documents))
len(ml_documents)

375

In [6]:
embeddings = []

for doc in ml_documents:
    # Transforming the title into an embedding using the model
    qa_text = f'{doc["question"]} {doc["text"]}'
    text_vector = embedding_model.encode(qa_text).tolist()
    embeddings.append(text_vector)

In [7]:
'''
Q2: What is the shape of X?
A: (375, 768)
'''
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

In [8]:
'''
Q3: What's the highest score in the results?
A: 0.65
'''
scores = X.dot(user_question_vector)
max(scores)

0.6506573240979582

In [9]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)
search_engine.search(user_question_vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [10]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [11]:
from tqdm import tqdm

search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)

relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    question_vector = embedding_model.encode(q['question'])
    results = search_engine.search(question_vector, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [02:20<00:00, 13.07it/s]


In [12]:
'''
Q4: What is the hitrate?
A: 0.93
'''
def hit_rate(relevance_total):
    hits = 0
    for line in relevance_total:
        if any(line):
            hits += 1
    return hits / len(relevance_total)

hit_rate(relevance_total)

0.9398907103825137

In [13]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '408019a87fd4', 'cluster_name': 'docker-cluster', 'cluster_uuid': '8fl16Pa_R9KyoHe0jirw7g', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "id": {"type": "text"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [22]:
index_name = "ml-course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml-course-questions'})

In [21]:
for doc in ml_documents:
    qa_text = f'{doc["question"]} {doc["text"]}'
    doc['text_vector'] = embedding_model.encode(qa_text).tolist()

In [23]:
for doc in ml_documents:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [31]:
'''
Q5: ID with the highest score?
A: 'ee58a693'
'''
query = {
    "field": "text_vector",
    "query_vector": user_question_vector,
    "k": 5,
    "num_candidates": 1000
}
results = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "id"])
results['hits']['hits'][0]['_source']['id']

'ee58a693'

In [32]:
from tqdm import tqdm

es_relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    vector_search_term = embedding_model.encode(q['question'])
    query = {
        "field": "text_vector",
        "query_vector": vector_search_term,
        "k": 5,
        "num_candidates": 1000
    }
    results = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "id"])
    relevance = [d['_source']['id'] == doc_id for d in results['hits']['hits']]
    es_relevance_total.append(relevance)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:08<00:00, 26.65it/s]


In [33]:
'''
Q6: What is the hitrate?
A: 0.93
'''
hit_rate(es_relevance_total)

0.9398907103825137